######data cleaning and EDA
categorical encoding (airline, source, destination) 
######EDA
data viz 
######Feature and target engineering
######ML modeling
######Conclusion --perform testing


In [46]:

#import missingno as msno
import numpy as np
import pandas as pd
#from plotnine import ggplot, aes, geom_density, geom_line, geom_point, ggtitle

from sklearn.model_selection import train_test_split
from sklearn.compose import TransformedTargetRegressor
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA

#from category_encoders.ordinal import OrdinalEncoder
#from feature_engine.encoding import RareLabelEncoder

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline

In [47]:
df = pd.read_csv('data/flights.csv')


In [48]:
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional Info,Price,Unnamed: 11,Unnamed: 12
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3898,NaN,NaN
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7663,NaN,NaN
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13883,NaN,NaN
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6219,NaN,NaN
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13303,NaN,NaN


In [49]:
df.shape

(10683, 13)

In [50]:
##drop unnamed columns
df.drop(['Unnamed: 11'], axis =1, inplace=True)
df.drop(['Unnamed: 12'], axis=1, inplace=True)

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


**Convert dtype of Date_of_Journay, Dep_Time and Arrival_Time to DateTime type**

In [52]:
###convert time to DateTime type
df['Date_of_Journey']=pd.to_datetime(df['Date_of_Journey'])
df['Dep_Time']=pd.to_datetime(df['Dep_Time'])
df['Arrival_Time']=pd.to_datetime(df['Arrival_Time'])

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Airline          10683 non-null  object        
 1   Date_of_Journey  10683 non-null  datetime64[ns]
 2   Source           10683 non-null  object        
 3   Destination      10683 non-null  object        
 4   Route            10682 non-null  object        
 5   Dep_Time         10683 non-null  datetime64[ns]
 6   Arrival_Time     10683 non-null  datetime64[ns]
 7   Duration         10683 non-null  object        
 8   Total_Stops      10682 non-null  object        
 9   Additional Info  10683 non-null  object        
 10  Price            10683 non-null  int64         
dtypes: datetime64[ns](3), int64(1), object(7)
memory usage: 918.2+ KB


In [54]:
#divide years, months, days of datetime dtype
df['journey_year']=df['Date_of_Journey'].dt.year
df['journey_month']=df['Date_of_Journey'].dt.month
df['journey_day']=df['Date_of_Journey'].dt.day

In [55]:
#drop Date_of_Journey
df.drop(['Date_of_Journey'], axis=1, inplace=True)

In [56]:
#divide dep_time to two columns: hr and min 
df['dep_hour']=df['Dep_Time'].dt.hour
df['dep_min']=df['Dep_Time'].dt.minute

In [57]:
#drop Dep_time
df.drop(['Dep_Time'], axis=1, inplace=True)

In [58]:
###devide arrival date
df['arrival_year']=df['Arrival_Time'].dt.year
df['arrival_month']=df['Arrival_Time'].dt.month
df['arrival_day']=df['Arrival_Time'].dt.day

In [59]:
##devide arrival time 
df['arrival_hour']=df['Arrival_Time'].dt.hour
df['arrival_min']=df['Arrival_Time'].dt.minute

In [60]:
#drop Arrival_Time
df.drop(['Arrival_Time'], axis=1, inplace=True)

In [61]:
#divide Duration to two columns: hours and mins
duration = list(df["Duration"]) #list Duration so we can spply split() and referencec it 
duration_hours = [] #create empty lists to append later with values from Duration
duration_mins = []



for i in range(len(duration)):
    duration_hours.append(duration[i].split(sep = "h")[0])
    duration_mins.append(duration[i].split(sep = "m")[0].split()[-1])

In [104]:
def get_hour(x):
    if x.find('h') == -1:
        return 0
    return x.split(sep = "h")[0]
def get_min(x):
    if x.find('m') == -1:
        return 0
    return x.split(sep = 'm')[0].split()[-1]

In [62]:
# ##adding duration_hours and duration_mins to df
# df['duration_hours']=duration_hours
# df['duration_mins']=duration_mins

In [105]:
df['duration_hours'] = df['Duration'].apply(get_hour)
df['duration_mins'] = df['Duration'].apply(get_min)

In [63]:
#drop Duration
df.drop(['Duration'], axis=1, inplace=True)

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Source           10683 non-null  object
 2   Destination      10683 non-null  object
 3   Route            10682 non-null  object
 4   Duration         10683 non-null  object
 5   Total_Stops      10682 non-null  object
 6   Additional Info  10683 non-null  object
 7   Price            10683 non-null  int64 
 8   journey_year     10683 non-null  int64 
 9   journey_month    10683 non-null  int64 
 10  journey_day      10683 non-null  int64 
 11  dep_hour         10683 non-null  int64 
 12  dep_min          10683 non-null  int64 
 13  arrival_year     10683 non-null  int64 
 14  arrival_month    10683 non-null  int64 
 15  arrival_day      10683 non-null  int64 
 16  arrival_hour     10683 non-null  int64 
 17  arrival_min      10683 non-null

**Handling null values**

In [65]:
#check for null values
df.isnull().sum()

Airline            0
Source             0
Destination        0
Route              1
Duration           0
Total_Stops        1
Additional Info    0
Price              0
journey_year       0
journey_month      0
journey_day        0
dep_hour           0
dep_min            0
arrival_year       0
arrival_month      0
arrival_day        0
arrival_hour       0
arrival_min        0
duration_hours     0
duration_mins      0
dtype: int64

In [66]:
#check the instance with NaN
df1 = df[df.isna().any(axis=1)]
df1

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional Info,Price,journey_year,journey_month,journey_day,dep_hour,dep_min,arrival_year,arrival_month,arrival_day,arrival_hour,arrival_min,duration_hours,duration_mins
9039,Air India,Delhi,Cochin,NaN,23h 40m,NaN,No info,7481,2019,6,5,9,45,2022,5,7,9,25,23,40


In [67]:
#drop the instance with NaN
df.dropna(inplace=True)

In [68]:
#renaming Additional Info column to be able to access .value_counts()
df.rename({'Additional Info': 'additional_info'}, axis=1, inplace=True)

In [69]:
#the feature [additional_info] contains 'No info' which counts as nulls 
df.additional_info.value_counts()


No info                         8344
In-flight meal not included     1982
No check-in baggage included     320
1 Long layover                    19
Change airports                    7
Business class                     4
No Info                            3
1 Short layover                    1
Red-eye flight                     1
2 Long layover                     1
Name: additional_info, dtype: int64

In [70]:
#since 'No info' represent approximately 76% of the column, and it appears to be missing at random, we'll drop the column
df.drop(['additional_info'], axis=1, inplace=True)

**Check for duplicates**

In [71]:
#check for duplicates
df.duplicated().value_counts()

False    10462
True       220
dtype: int64

In [72]:
#view duplicated rows
df[df.duplicated()]

,Airline,Source,Destination,Route,Duration,Total_Stops,Price,journey_year,journey_month,journey_day,dep_hour,dep_min,arrival_year,arrival_month,arrival_day,arrival_hour,arrival_min,duration_hours,duration_mins
683,Jet Airways,Delhi,Cochin,DEL → NAG → BOM → COK,13h 50m,2 stops,13377,2019,1,6,14,35,2022,6,2,4,25,13,50
1061,Air India,Delhi,Cochin,DEL → GOI → BOM → COK,21h 15m,2 stops,10232,2019,5,21,22,0,2022,5,22,19,15,21,15
1348,Air India,Delhi,Cochin,DEL → HYD → BOM → COK,26h,2 stops,12393,2019,5,18,17,15,2022,5,19,19,15,26,26h
1418,Jet Airways,Delhi,Cochin,DEL → JAI → BOM → COK,22h 55m,2 stops,10369,2019,6,6,5,30,2022,6,7,4,25,22,55
1674,IndiGo,Banglore,New Delhi,BLR → DEL,2h 55m,non-stop,7304,2019,3,24,18,25,2022,2,15,21,20,2,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10594,Jet Airways,Delhi,Cochin,DEL → AMD → BOM → COK,13h 30m,2 stops,12820,2019,6,27,23,5,2022,6,28,12,35,13,30
10616,Jet Airways,Delhi,Cochin,DEL → JAI → BOM → COK,26h 55m,2 stops,13015,2019,1,6,9,40,2022,6,2,12,35,26,55
10634,Jet Airways,Delhi,Cochin,DEL → JAI → BOM → COK,26h 55m,2 stops,11734,2019,6,6,9,40,2022,6,7,12,35,26,55
10672,Jet Airways,Delhi,Cochin,DEL → AMD → BOM → COK,19h 55m,2 stops,11151,2019,6,27,23,5,2022,6,28,19,0,19,55


In [73]:
#drop duplicated rows
df.drop_duplicates(keep='first',inplace=True)

**Encoding categorical vaiables**

In [74]:
#encode Total_Stops
df['Total_Stops'].replace({'non-stop':0, '1 stop': 1, '2 stops': 2, '3 stops':3, '4 stops':4}, inplace=True)

In [75]:
df.Total_Stops.value_counts()

1    5625
0    3475
2    1318
3      43
4       1
Name: Total_Stops, dtype: int64

In [76]:
df['Airline'].unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [77]:
df['Airline'].replace({'IndiGo':0, 'Air India': 1, 'Jet Airways': 2, 'SpiceJet':3, 'Multiple carriers':4, 'GoAir':5, 'Vistara':6, 'Air Asia':7, 'Vistara Premium economy':8, 'Jet Airways Business':9, 'Multiple carriers Premium economy':10, 'Trujet':11}, inplace=True)

In [78]:
df.Airline.dtype

dtype('int64')

In [79]:
df['Source'].unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

In [80]:
df['Source'].replace({'Banglore':0, 'Kolkata': 1, 'Delhi': 2, 'Chennai':3, 'Mumbai':4}, inplace=True)

In [81]:
df.Source.dtype

dtype('int64')

In [82]:
df['Destination'].unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

In [83]:
df['Destination'].replace({'Banglore':0, 'Kolkata': 1, 'Delhi': 2, 'New Delhi':3, 'Hyderabad':4, 'Cochin':5}, inplace=True)

In [84]:
df.Destination.dtype

dtype('int64')

In [103]:
df.to_csv('data/flights_cleaned.csv')